In [11]:
# import cv2
# import os

# def play_video_fitting_screen(video_path, save_folder):
#     cap = cv2.VideoCapture(video_path)

#     # 获取视频的原始宽高
#     video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

#     # 获取屏幕的宽高
#     screen_width = 1920
#     screen_height = 1080

#     # 计算缩放比例，确保视频完整显示
#     scale_width = screen_width / video_width
#     scale_height = screen_height / video_height
#     scale = min(scale_width, scale_height)  # 选择最小的缩放比例，确保适配屏幕

#     # 计算缩放后的窗口大小
#     new_width = int(video_width * scale)
#     new_height = int(video_height * scale)

#     # 计算居中的位置
#     x_pos = (screen_width - new_width) // 2
#     y_pos = (screen_height - new_height) // 2

#     # 创建窗口
#     window_name = "Video Playback"
#     cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

#     # 设置窗口大小
#     cv2.resizeWindow(window_name, new_width, new_height)

#     # 移动窗口到居中位置
#     cv2.moveWindow(window_name, x_pos, y_pos)

#     # 确保保存文件夹存在
#     if not os.path.exists(save_folder):
#         os.makedirs(save_folder)

#     frame_index = 0  # 帧编号

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # 缩放帧，使其适应窗口大小
#         frame_resized = cv2.resize(frame, (new_width, new_height))

#         cv2.imshow(window_name, frame_resized)
#         key = cv2.waitKey(30)

#         if key == ord('q'):  # 按 'q' 退出
#             break
#         elif key == ord('s'):  # 按 's' 保存当前帧
#             filename = f"frame_{frame_index}.jpg"
#             file_path = os.path.join(save_folder, filename)
#             success = cv2.imwrite(file_path, frame)  # 这里保存原始尺寸的帧，而不是缩放后的
#             if success:
#                 print(f"帧已保存: {file_path}")
#             else:
#                 print(f"保存帧失败: {file_path}")

#         frame_index += 1  # 增加帧编号

#     cap.release()
#     cv2.destroyAllWindows()

# # 示例调用
# video_path = r'C:\Users\My_xuan\Desktop\Align\Video\chess\3.MP4'
# save_folder = r'C:\Users\My_xuan\Desktop\Align\Video\chess\3'
# play_video_fitting_screen(video_path, save_folder)


帧已保存: C:\Users\My_xuan\Desktop\Align\Video\chess\3\frame_422.jpg
帧已保存: C:\Users\My_xuan\Desktop\Align\Video\chess\3\frame_623.jpg


### 指定棋盘格图像的路径，确保全是jpg图片运行下面代码，得到npz畸变数据。

In [4]:
import cv2
import glob
import numpy as np


class ChessCalibrate:
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    def __init__(self, checkerboard: tuple):
        self.CHECKERBOARD = checkerboard  #棋盘格的尺寸是17，17
        self.imgpoints = []
        self.objpoints = []
        self.objp = np.zeros((1, self.CHECKERBOARD[0] * self.CHECKERBOARD[1], 3), np.float32)   #棋盘格的3D坐标
        self.objp[0, :, :2] = np.mgrid[
            0 : self.CHECKERBOARD[0], 0 : self.CHECKERBOARD[1]
        ].T.reshape(-1, 2)
        self.imgsize=None

    def addImg(self, img: np.array):
        if self.imgsize is None:
            self.imgsize=img.shape
        retval, corners = cv2.findChessboardCorners(img, self.CHECKERBOARD)
        if retval:
            corners2 = cv2.cornerSubPix(
                img, corners, (11, 11), (-1, -1), self.criteria
            )
            self.objpoints.append(self.objp)
            self.imgpoints.append(corners2)
        else:
            print(f"棋盘格检测失败，跳过图像：{img}")

    def calibrate(self):
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
            self.objpoints, self.imgpoints, self.imgsize[::-1], None, None
        )
        return ret, mtx, dist

    def __len__(self):
        return len(self.imgpoints)


if __name__ == "__main__":
    CHECKERBOARD = (17, 17)
    tool = ChessCalibrate(CHECKERBOARD)
    imglist = glob.glob("./Video/chess/all/*.jpg")   #注意一下是jpg的全部图片还是bmp的全部图片
    for img in imglist:
        gray = cv2.imread(img, 0)
        tool.addImg(gray)
        print("Image:",len(tool))
    ret, mtx, dist = tool.calibrate()
    np.savez("./output/distort", mtx=mtx, dist=dist)
    # Read Saved File
    # distort = np.load("distort.npz")
    # print(distort['mtx'])
    # print(distort['dist'])





Image: 1
棋盘格检测失败，跳过图像：[[104 105 105 ...  25  25  25]
 [105 105 105 ...  25  25  25]
 [104 104 105 ...  25  25  25]
 ...
 [  2   2   2 ... 113 112 112]
 [  2   2   2 ... 113 113 112]
 [  2   2   2 ... 113 113 113]]
Image: 1
Image: 2
Image: 3
Image: 4
Image: 5
Image: 6
棋盘格检测失败，跳过图像：[[  5   5   5 ... 135 133 133]
 [  5   5   5 ... 136 133 134]
 [  5   5   5 ... 135 133 133]
 ...
 [  2   2   2 ... 118 118 118]
 [  2   2   2 ... 118 118 118]
 [  2   2   2 ... 118 118 118]]
Image: 6
Image: 7
Image: 8
Image: 9
Image: 10


# 抽取图像1，同时播放抽取图像,自行选择

In [1]:
import cv2
import os

def play_dual_video(infrared_path, visible_path, save_folder):
    cap_infrared = cv2.VideoCapture(infrared_path)
    cap_visible = cv2.VideoCapture(visible_path)

    # 获取两个视频的原始宽高（假设两者相同）
    video_width = int(cap_visible.get(cv2.CAP_PROP_FRAME_WIDTH))
    video_height = int(cap_visible.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 获取屏幕的宽高
    screen_width = 1920
    screen_height = 1080

    # 计算缩放比例，确保两个视频完整显示
    scale_width = screen_width / (2 * video_width)  # 两个视频并排显示
    scale_height = screen_height / video_height
    scale = min(scale_width, scale_height)  # 选择最小的缩放比例，确保适配屏幕

    # 计算缩放后的窗口大小
    new_width = int(video_width * scale)
    new_height = int(video_height * scale)

    # 计算窗口位置
    x_pos_left = (screen_width // 2) - new_width
    x_pos_right = (screen_width // 2)
    y_pos = (screen_height - new_height) // 2

    # 创建窗口
    window_name_infrared = "Infrared Video"
    window_name_visible = "Visible Video"
    cv2.namedWindow(window_name_infrared, cv2.WINDOW_NORMAL)
    cv2.namedWindow(window_name_visible, cv2.WINDOW_NORMAL)

    # 设置窗口大小
    cv2.resizeWindow(window_name_infrared, new_width, new_height)
    cv2.resizeWindow(window_name_visible, new_width, new_height)

    # 移动窗口到指定位置
    cv2.moveWindow(window_name_infrared, x_pos_left, y_pos)
    cv2.moveWindow(window_name_visible, x_pos_right, y_pos)

    # 确保保存文件夹存在
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # 获取视频名称
    video_a_name = os.path.basename(infrared_path).split('.')[0]  # 获取红外视频的文件名（不包含扩展名）
    video_b_name = os.path.basename(visible_path).split('.')[0]  # 获取可见光视频的文件名（不包含扩展名）

    frame_index = 0  # 帧编号

    while cap_infrared.isOpened() and cap_visible.isOpened():
        ret_infrared, frame_infrared = cap_infrared.read()
        ret_visible, frame_visible = cap_visible.read()

        if not ret_infrared or not ret_visible:
            break

        # 缩放帧，使其适应窗口大小
        frame_infrared_resized = cv2.resize(frame_infrared, (new_width, new_height))
        frame_visible_resized = cv2.resize(frame_visible, (new_width, new_height))

        cv2.imshow(window_name_infrared, frame_infrared_resized)
        cv2.imshow(window_name_visible, frame_visible_resized)

        key = cv2.waitKey(30)

        if key == ord('q'):  # 按 'q' 退出
            break
        elif key == ord('s'):  # 按 's' 保存当前帧
            # 创建 infrared 和 visible 文件夹
            infrared_folder = os.path.join(save_folder, "infrared")
            visible_folder = os.path.join(save_folder, "visible")
            if not os.path.exists(infrared_folder):
                os.makedirs(infrared_folder)
            if not os.path.exists(visible_folder):
                os.makedirs(visible_folder)

            # 根据命名规则生成文件路径
            file_path_a = os.path.join(infrared_folder, f"{video_a_name}_v{frame_index}.jpg")
            file_path_b = os.path.join(visible_folder, f"{video_b_name}_i{frame_index}.jpg")

            success_a = cv2.imwrite(file_path_a, frame_infrared)
            success_b = cv2.imwrite(file_path_b, frame_visible)

            if success_a and success_b:
                print(f"帧已保存: {file_path_a} 和 {file_path_b}")
            else:
                print("保存帧失败")

        frame_index += 1  # 增加帧编号

    cap_infrared.release()
    cap_visible.release()
    cv2.destroyAllWindows()

def process_multiple_videos(save_folder):
    # 对视频 1.MP4 到 3.MP4 进行循环处理
    for i in range(1, 4):
        infrared_video = os.path.join('./Video/infrared', f'{i}.MP4')
        visible_video = os.path.join('./Video/visible', f'{i}.MP4')

        print(f"正在处理视频 {i}.MP4 和 {i}.MP4")
        play_dual_video(infrared_video, visible_video, save_folder)

# 调用多个视频处理函数
save_folder = './output/img'
process_multiple_videos(save_folder)


正在处理视频 1.MP4 和 1.MP4
正在处理视频 2.MP4 和 2.MP4
正在处理视频 3.MP4 和 3.MP4


# 抽取图像2

In [2]:
import cv2
import os

def extract_frames_dual(video_a_path, video_b_path, save_folder, frame_interval=100):
    # 打开视频
    cap_a = cv2.VideoCapture(video_a_path)
    cap_b = cv2.VideoCapture(video_b_path)

    # 检查视频是否成功打开
    if not cap_a.isOpened() or not cap_b.isOpened():
        print("无法打开视频文件，请检查文件路径。")
        return

    # 获取视频文件名（去掉路径和后缀）
    video_a_name = os.path.splitext(os.path.basename(video_a_path))[0]  # 例如 "1"
    video_b_name = os.path.splitext(os.path.basename(video_b_path))[0]  # 例如 "2"

    # 创建子文件夹 'infrared' 和 'visible'，确保保存文件夹存在
    infrared_folder = os.path.join(save_folder, "infrared")
    visible_folder = os.path.join(save_folder, "visible")
    
    if not os.path.exists(infrared_folder):
        os.makedirs(infrared_folder)
    if not os.path.exists(visible_folder):
        os.makedirs(visible_folder)

    frame_index = 0  # 帧计数器
    save_index_a = 1  # 红外（i）保存编号
    save_index_b = 1  # 可见光（v）保存编号

    while cap_a.isOpened() and cap_b.isOpened():
        ret_a, frame_a = cap_a.read()
        ret_b, frame_b = cap_b.read()

        if not ret_a or not ret_b:
            break  # 视频读取结束

        # 每隔 frame_interval 帧保存一次
        if frame_index % frame_interval == 0:
            # 生成文件名
            file_path_a = os.path.join(infrared_folder, f"{video_a_name}_i{save_index_a}.jpg")
            file_path_b = os.path.join(visible_folder, f"{video_b_name}_v{save_index_b}.jpg")
            # 保存图片
            success_a = cv2.imwrite(file_path_a, frame_a)
            success_b = cv2.imwrite(file_path_b, frame_b)
            if success_a and success_b:
                print(f"帧已保存: {file_path_a} 和 {file_path_b}")
            else:
                print("保存帧失败")
            # 递增保存编号
            save_index_a += 1
            save_index_b += 1
        frame_index += 1  # 增加帧编号

    # 释放资源
    cap_a.release()
    cap_b.release()
    print("视频处理完成，所有帧已提取！")

# 示例调用
def extract_from_multiple_videos():
    save_folder = './output/img_sameinterval'
    
    # 循环处理视频1到视频3
    for i in range(1, 4):
        video_a = os.path.join('./Video/infrared', f'{i}.MP4')
        video_b = os.path.join('./Video/visible', f'{i}.MP4')

        print(f"正在处理视频 {i}.MP4 和 {i}.MP4")
        extract_frames_dual(video_a, video_b, save_folder)

# 调用多视频提取方法
extract_from_multiple_videos()


正在处理视频 1.MP4 和 1.MP4
帧已保存: ./output/img_sameinterval\infrared\1_i1.jpg 和 ./output/img_sameinterval\visible\1_v1.jpg
视频处理完成，所有帧已提取！
正在处理视频 2.MP4 和 2.MP4
帧已保存: ./output/img_sameinterval\infrared\2_i1.jpg 和 ./output/img_sameinterval\visible\2_v1.jpg
视频处理完成，所有帧已提取！
正在处理视频 3.MP4 和 3.MP4
帧已保存: ./output/img_sameinterval\infrared\3_i1.jpg 和 ./output/img_sameinterval\visible\3_v1.jpg
视频处理完成，所有帧已提取！


### 对指定img进行去畸变

In [5]:
# import numpy as np
# import cv2

# # 加载标定参数
# distort = np.load("./output/distort.npz")
# print("内参矩阵 (mtx):")
# print(distort['mtx'])
# print("畸变系数 (dist):")
# print(distort['dist'])

# distort = np.load("./output/distort.npz")
# mtx = distort['mtx']
# dist = distort['dist']

# # 读取待矫正的图像
# img = cv2.imread('./output/img/img/1_i43.jpg')

# # 计算矫正映射并去畸变
# h, w = img.shape[:2]  # 图像高度和宽度
# new_camera_mtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
# # 去畸变
# dst = cv2.undistort(img, mtx, dist, None, new_camera_mtx)

# # 裁剪图像
# # x, y, w, h = roi
# # dst = dst[y:y+h, x:x+w]

# # 使用 ROI 裁剪无效区域（可选）
# x, y, w, h = roi
# dst_cropped = dst[y:y+h, x:x+w]

# # 保存矫正后的图像
# # cv2.imwrite('./img_undistorted.jpg', dst)

# # 可视化去畸变效果
# cv2.imshow("Original Image", img)
# cv2.imshow("Undistorted Image", dst)
# cv2.imshow("Undistorted Image (Cropped)", dst_cropped)  # 裁剪后的图像
# cv2.waitKey(0)
# cv2.destroyAllWindows()


内参矩阵 (mtx):
[[936.31174104   0.         662.33070018]
 [  0.         939.38368853 357.85592349]
 [  0.           0.           1.        ]]
畸变系数 (dist):
[[ 4.20102316e-01 -2.15337317e+00 -2.13957494e-04 -3.25949428e-03
   3.56659438e+00]]


# 保存

In [3]:
import os
import cv2
import numpy as np
from typing import Union

class HomographyInit(object):
    """
    通过手动特征点匹配获取初始单应性矩阵
    """

    @staticmethod
    def run(
        src: np.ndarray,
        dst: np.ndarray,
        save_path: Union[None, str] = None,
        show: bool = False,
    ):
        # 备份用于计算变换的原始图片
        src_copy = src.copy()
        dst_copy = dst.copy()
        # 选取点的列表
        list_src = []
        list_dst = []



        def get_srcpoint(event, x, y, flags, param):
            if event == cv2.EVENT_LBUTTONDBLCLK:
                list_xy = [x, y]

                cv2.circle(src, (x, y), 2, (255, 0, 0), thickness=-1)
                cv2.putText(
                    src,
                    "%d,%d" % (x, y),
                    (x, y),
                    cv2.FONT_HERSHEY_PLAIN,
                    2.0,
                    (255, 255, 0),
                    thickness=1,
                )
                cv2.imshow("original_img", src)
                list_src.append(list_xy)

        # def get_srcpoint(event, x, y, flags, param):
        #     if event == cv2.EVENT_LBUTTONDBLCLK:
        #         list_xy = [x, y]

        #         # 只在备份图像上绘制，不影响原始图像
        #         temp_src = src.copy()
        #         cv2.circle(temp_src, (x, y), 2, (255, 0, 0), thickness=-1)
        #         cv2.putText(
        #             temp_src,
        #             "%d,%d" % (x, y),
        #             (x, y),
        #             cv2.FONT_HERSHEY_PLAIN,
        #             2.0,
        #             (255, 255, 0),
        #             thickness=1,
        #         )
        #         cv2.imshow("original_img", temp_src)
        #         list_src.append(list_xy)


        def get_dstpoint(event, x, y, flags, param):
            if event == cv2.EVENT_LBUTTONDBLCLK:
                list_xy = [x, y]
                cv2.circle(dst, (x, y), 2, (255, 0, 0), thickness=-1)
                cv2.putText(
                    dst,
                    "%d,%d" % (x, y),
                    (x, y),
                    cv2.FONT_HERSHEY_PLAIN,
                    1.0,
                    (255, 255, 0),
                    thickness=1,
                )
                cv2.imshow("target", dst)
                list_dst.append(list_xy)


        # def get_dstpoint(event, x, y, flags, param):
        #     if event == cv2.EVENT_LBUTTONDBLCLK:
        #         list_xy = [x, y]
                
        #         # 只在备份图像上绘制，不影响原始图像
        #         temp_dst = dst.copy()
        #         cv2.circle(temp_dst, (x, y), 2, (255, 0, 0), thickness=-1)
        #         cv2.putText(
        #             temp_dst,
        #             "%d,%d" % (x, y),
        #             (x, y),
        #             cv2.FONT_HERSHEY_PLAIN,
        #             1.0,
        #             (255, 255, 0),
        #             thickness=1,
        #         )
        #         cv2.imshow("target", temp_dst)
        #         list_dst.append(list_xy)


        cv2.namedWindow("original_img", cv2.WINDOW_NORMAL)
        cv2.namedWindow("target", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("original_img", 1280, 960)
        cv2.resizeWindow("target", 1280, 960)
        cv2.setMouseCallback("original_img", get_srcpoint)
        cv2.setMouseCallback("target", get_dstpoint)
        cv2.imshow("original_img", src)
        cv2.imshow("target", dst)
        cv2.waitKey()
        cv2.destroyAllWindows()
        
        # 转换点坐标为 NumPy 格式
        src_pts = np.float32(list_src)
        dst_pts = np.float32(list_dst)

        if len(src_pts) != len(dst_pts) or len(src_pts) < 4:
            raise ValueError("选取点不足或不匹配")

        # 计算单应性矩阵
        H, mask = cv2.findHomography(src_pts, dst_pts)

        if save_path is not None:
            np.save(save_path, H)
            print(f"Homography matrix saved at {save_path}")

        if show:
            HomographyInit.show(src_copy, dst_copy, H, src_pts, dst_pts)

        return H, src_pts, dst_pts  # 返回特征点，以便后续绘制

    @staticmethod
    def show(src, dst, H, src_pts, dst_pts, alpha=0.7, beta=0.3, borderValue=0):
        """ 显示融合后的图像，同时绘制特征点 """
        transformed_src = cv2.warpPerspective(src, H, dst.shape[:2][::-1], borderValue=borderValue)
        show_img = cv2.addWeighted(transformed_src, alpha, dst, beta, 1)

        # 画特征点
        for p1, p2 in zip(src_pts, dst_pts):
            p1 = tuple(map(int, p1))
            p2 = tuple(map(int, p2))
            cv2.circle(show_img, p2, 5, (0, 0, 255), -1)  # 目标图像上的点
            cv2.line(show_img, p1, p2, (255, 255, 0), 1)

        return show_img

    @staticmethod
    def fuse_images(img1, img2, alpha=0.5, beta=0.5):
        """ 融合两张图像 """
        return cv2.addWeighted(img1, alpha, img2, beta, 0)
    
    
if __name__ == "__main__":
    # 加载相机标定参数
    distort = np.load("./output/distort.npz")
    mtx = distort['mtx']
    dist = distort['dist']

    # 文件夹路径
    src_folder = './output/img/visible'
    dst_folder = './output/img/infrared'
    save_folder = './output/img_new'

    # 创建保存文件夹
    infrared_output = os.path.join(save_folder, "infrared")
    visible_output = os.path.join(save_folder, "visible")
    comparasion_output = os.path.join(save_folder, "comparasion")

    os.makedirs(infrared_output, exist_ok=True)
    os.makedirs(visible_output, exist_ok=True)
    os.makedirs(comparasion_output, exist_ok=True)

    # 获取图像文件
    visible_images = sorted([f for f in os.listdir(src_folder) if f.endswith('.jpg') or f.endswith('.png')])
    infrared_images = sorted([f for f in os.listdir(dst_folder) if f.endswith('.jpg') or f.endswith('.png')])

    for visible_image, infrared_image in zip(visible_images, infrared_images):
        # 读取图像
        src = cv2.imread(os.path.join(src_folder, visible_image))
        dst = cv2.imread(os.path.join(dst_folder, infrared_image))
        
        if src is None or dst is None:
            print(f"Error loading images: {visible_image}, {infrared_image}")
            continue

        # 去畸变
        src = cv2.undistort(src, mtx, dist, None, None)


        temp_src = src.copy()
        temp_dst = dst.copy()

        # 计算单应性变换
        H, src_pts, dst_pts = HomographyInit.run(src, dst, save_path=None, show=False)


        warped_src = cv2.warpPerspective(src, H, (dst.shape[1], dst.shape[0]))
        
        warped_temp_src = cv2.warpPerspective(temp_src, H, (temp_dst.shape[1], temp_dst.shape[0]))

        # # 生成对比图（包含特征点）
        # comp_img = HomographyInit.show(src, dst, H, src_pts, dst_pts)

        
        # 调整 src 和 dst 的大小，使其一致 
        target_size = (min(src.shape[1], dst.shape[1]), min(src.shape[0], dst.shape[0]))
        warped_src_resized = cv2.resize(warped_src, target_size)
        dst_resized = cv2.resize(dst, target_size)
        # 水平拼接两张图片
        comp_img = np.hstack((warped_src_resized, dst_resized))


        # 生成融合图
        fused_img = HomographyInit.fuse_images(warped_src, dst, alpha=0.7, beta=0.3)


        # 生成融合图
        fused_temp_img = HomographyInit.fuse_images(warped_temp_src, temp_dst, alpha=0.7, beta=0.3)



        # 保存图像
        cv2.imwrite(os.path.join(visible_output, visible_image), warped_temp_src)  # 变换后的可见光图像
        cv2.imwrite(os.path.join(infrared_output, infrared_image), temp_dst)  # 原始红外图像
        cv2.imwrite(os.path.join(comparasion_output, f"comp_{visible_image}"), comp_img)  # 对比图
        cv2.imwrite(os.path.join(comparasion_output, f"fused_{visible_image}"), fused_img)  # 融合图
        cv2.imwrite(os.path.join(comparasion_output, f"fused2_{visible_image}"), fused_temp_img)  # 融合图

        print(f"Processed: {visible_image} and {infrared_image}")

    print("Image processing complete!")

Processed: 1_i32.jpg and 1_v32.jpg
Processed: 2_i17.jpg and 2_v17.jpg


ValueError: 选取点不足或不匹配